# Korean LLM (Large Language Model) Serving on SageMaker with AWS Large Model Container DLC
---

### Model: [KoAlpaca-KoRWKV-6B](https://huggingface.co/beomi/KoAlpaca-KoRWKV-6B)

한국어 LLM 모델 SageMaker 서빙 핸즈온 (LLM을 모델을 S3에 복사 후 S3에서 s5cmd로 배포)

- LLM GitHub: https://github.com/Beomi/KoAlpaca
- [AWS Blog: Deploy large models on Amazon SageMaker using DJLServing and DeepSpeed model parallel inference](https://aws.amazon.com/ko/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference)

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates')

In [ ]:
!pip install -qU boto3 huggingface_hub sagemaker langchain deepspeed 

In [ ]:
import os
import sagemaker, boto3, jinja2
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3")  # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime")  # client to intreract with SageMaker Endpoints
jinja_env = jinja2.Environment()  # jinja environment to generate model configuration templates

<br>

## 1. Download LLM model and upload it to S3
---

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

model_id = "beomi/KoAlpaca-KoRWKV-6B"
model_prefix = model_id.split('/')[-1].replace('.', '-')
model_tar_dir = f"/home/ec2-user/SageMaker/models/{model_prefix}"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.txt", "*.model", "*.safetensors"]
#allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

if not os.path.isdir(model_tar_dir):
    os.makedirs(model_tar_dir, exist_ok=True)
    # - Leverage the snapshot library to donload the model since the model is stored in repository using LFS    
    snapshot_download(
        repo_id=model_id,
        local_dir=str(model_tar_dir), 
        local_dir_use_symlinks=False,        
        allow_patterns=allow_patterns,
        cache_dir="/home/ec2-user/SageMaker/"        
    )

bucket_prefix = 'ko-llms/serving'    
s3_code_prefix = f"{bucket_prefix}/{model_prefix}/code"  # folder within bucket where code artifact will go
s3_model_prefix = f"{bucket_prefix}/{model_prefix}/model"  # folder where model checkpoint will go
s3_model_artifact = f"s3://{bucket}/{s3_model_prefix}"

print(f"S3 code prefix \n {s3_code_prefix}")
print(f"S3 model prefix: \n {s3_model_prefix}")
print(f"S3 model artifact path: \n {s3_model_artifact}")

In [ ]:
%%bash
aws configure set default.s3.max_concurrent_requests 100
aws configure set default.s3.max_queue_size 10000
aws configure set default.s3.multipart_threshold 1GB
aws configure set default.s3.multipart_chunksize 64MB

In [ ]:
!aws s3 sync {model_tar_dir} {s3_model_artifact}
print(f"Model uploaded to --- > {s3_model_artifact}")
print(f"We will set option.s3url={s3_model_artifact}")

<br>

## 2. Model Serving Scripts
---
### Create `serving.properties`

이 설정 파일은 어떤 추론 최적화 라이브러리를 사용할지, 어떤 설정을 사용할지 DJL Serving에 알려주는 설정 파일입니다. 필요에 따라 적절한 구성을 설정할 수 있습니다.

모델이 레이어에 따라 분할되는 파이프라인 병렬화(Pipeline Parallelism)를 사용하는 허깅페이스 Accelerate와 달리, DeepSpeed는 각 레이어(텐서)가 여러 디바이스에 걸쳐 샤딩되는 텐서 병렬화(Tensor Parallelism)를 사용합니다. 파이프라인 병렬 처리 접근 방식에서는 데이터가 각 GPU 장치를 통해 순차적으로 흐르지만, 텐서 병렬 처리는 데이터가 모든 GPU 장치로 전송되어 각 GPU에서 부분적인 결과가 계산됩니다. 그런 다음 All-Gather 연산을 통해 부분 결과를 수집하여 최종 결과를 계산합니다. 따라서, 텐서 병렬화가 일반적으로 더 높은 GPU 활용률과 더 나은 성능을 제공합니다.

- `option.s3url` - 모델 파일의 위치를 지정합니다. 또는`option.model_id` 옵션을 대신 사용하여 허깅페이스 허브에서 모델을 지정할 수 있습니다(예: EleutherAI/gpt-j-6B). 그러면 허브에서 모델이 자동으로 다운로드됩니다. s3url 접근 방식은 자체 환경 내에서 모델 아티팩트를 호스팅할 수 있고 DJL 추론 컨테이너 내에서 최적화된 접근 방식을 활용하여 S3에서 호스팅 인스턴스로 모델을 전송함으로써 더 빠른 모델 배포가 가능합니다.

`serving.properties`의 일반적인 설정법과 자세한 내용은 https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-configuration.html 를 참조하세요.

<img src="../images/TensorShard.png" width="800"/>

In [ ]:
src_path = f"src/{model_prefix}"
!rm -rf {src_path}
os.makedirs(src_path, exist_ok=True)

In [ ]:
%%writefile {src_path}/serving.properties
option.s3url={{s3url}}

engine=DeepSpeed

# passing extra options to model.py or built-in handler
job_queue_size=100
batch_size=1
max_batch_delay=1
max_idle_time=60

# Built-in entrypoint
#option.entryPoint=djl_python.deepspeed

# Hugging Face model id
#option.model_id=EleutherAI/gpt-j-6B

# defines custom environment variables
#env=SERVING_NUMBER_OF_NETTY_THREADS=2

# Allows to load DeepSpeed workers in parallel
option.parallel_loading=true

# specify tensor parallel degree (number of partitions)
option.tensor_parallel_degree=4

# specify per model timeout
option.model_loading_timeout=600
#option.predict_timeout=240

# mark the model as failure after python process crashing 10 times
retry_threshold=0

option.task=text-generation

### serving.properties의 S3 경로 수정

In [ ]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path(f"{src_path}/serving.properties").open().read())
Path(f"{src_path}/serving.properties").open("w").write(template.render(s3url=s3_model_artifact))
!pygmentize {src_path}/serving.properties | cat -n

### Create model.py with custom inference code
빌트인 추론 코드로 no-code로 배포할 수도 있지만, 커스텀 추론 코드를 작성하는 것도 가능합니다.

In [ ]:
%%writefile {src_path}/model.py
from djl_python import Input, Output
import os
import deepspeed
import torch
import logging
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers import GPTNeoXLayer

predictor = None

def get_model(properties):
    
    tp_degree = properties["tensor_parallel_degree"]
    model_location = properties["model_dir"]
    if "model_id" in properties:
        model_location = properties["model_id"]
    task = properties["task"]
    
    logging.info(f"Loading model in {model_location}")    
    local_rank = int(os.getenv("LOCAL_RANK", "0"))

    tokenizer = AutoTokenizer.from_pretrained(model_location)

    model = AutoModelForCausalLM.from_pretrained(
        model_location,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    )
    
    model.requires_grad_(False)
    model.eval()
    
    ds_config = {
        "tensor_parallel": {"tp_size": tp_degree},
        "dtype": model.dtype,
        "injection_policy": {
            GPTNeoXLayer:('attention.dense', 'mlp.dense_4h_to_h')
        }
    }
    logging.info(f"Starting DeepSpeed init with TP={tp_degree}")        
    model = deepspeed.init_inference(model, ds_config)  
    
    generator = pipeline(
        task=task, model=model, tokenizer=tokenizer, device=local_rank
    )
    # https://huggingface.co/docs/hub/models-tasks
    return generator
    
def handle(inputs: Input) -> None:
    """
    inputs: Contains the configurations from serving.properties
    """    
    global predictor
    if not predictor:
        predictor = get_model(inputs.get_properties())

    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        logging.info("is_empty")
        return None

    data = inputs.get_as_json() #inputs.get_as_string()
    logging.info("data:", data)
    
    input_prompt, params = data["inputs"], data["parameters"]
    result = predictor(input_prompt, **params)
    logging.info("result:", result)

    return Output().add_as_json(result) #Output().add(result)

### Create the Tarball and then upload to S3

In [ ]:
!rm -rf model.tar.gz
!tar czvf model.tar.gz -C {src_path} .

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")
!rm -rf model.tar.gz

<br>

## 3. Serve LLM Model on SageMaker

---

### Create SageMaker Model

SageMaker 엔드포인트 생성 매개변수 VolumeSizeInGB를 지정할 때 마운트되는 Amazon EBS(Amazon Elastic Block Store) 볼륨에 /tmp를 매핑하기 때문에 컨테이너는 인스턴스의 `/tmp` 공간에 모델을 다운로드합니다. 이때 s5cmd (https://github.com/peak/s5cmd) 를 활용하므로 대용량 모델을 빠르게 다운로드할 수 있습니다.
볼륨 인스턴스와 함께 미리 빌드되어 제공되는 p4dn과 같은 인스턴스의 경우 컨테이너의 `/tmp`를 계속 활용할 수 있습니다. 

In [ ]:
from sagemaker.utils import name_from_base
from sagemaker import image_uris

img_uri = image_uris.retrieve(framework="djl-deepspeed", region=region, version="0.23.0")
model_name = name_from_base(f"{model_prefix}")
print(model_name)

model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": img_uri, "ModelDataUrl": s3_code_artifact},
)
model_arn = model_response["ModelArn"]
print(f"Created Model: {model_arn}")

### Create SageMaker Endpoint

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"
variant_name = "variant1"
instance_type = "ml.g5.12xlarge"
initial_instance_count = 1

prod_variants = [
    {
        "VariantName": "variant1",
        "ModelName": model_name,
        "InstanceType": instance_type,
        "InitialInstanceCount": initial_instance_count,
        # "ModelDataDownloadTimeoutInSeconds": 2400,
        "ContainerStartupHealthCheckTimeoutInSeconds": 600,
    }
]

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=prod_variants
)

endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {endpoint_response['EndpointArn']}")

엔드포인트가 생성되는 동안 아래의 문서를 같이 확인해 보세요.
- https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-dlc.html

In [ ]:
from IPython.display import display, HTML
def make_console_link(region, endpoint_name, task='[SageMaker LLM Serving]'):
    endpoint_link = f'<b> {task} <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">Check Endpoint Status</a></b>'   
    return endpoint_link

endpoint_link = make_console_link(region, endpoint_name)
display(HTML(endpoint_link))

In [ ]:
%%time 
from inference_lib import describe_endpoint, Prompter
describe_endpoint(endpoint_name)         

<br>

## 4. Inference
---

엔드포인트를 호출할 때 이 텍스트를 JSON 페이로드 내에 제공해야 합니다. 이 JSON 페이로드에는 length, sampling strategy, output token sequence restrictions을 제어하는 데 도움이 되는 원하는 추론 매개변수가 포함될 수 있습니다. 허깅페이스 트랜스포머 transformers 라이브러리에는 [사용 가능한 페이로드 매개변수](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig)의 전체 목록이 정의되어 있지만, 중요한 페이로드 매개변수는 다음과 같이 정의되어 있습니다:

* **do_sample (`bool`)** – logits sampling 활성화
* **max_new_tokens (`int`)** – 생성된 토큰의 최대 수
* **best_of (`int`)** – best_of 개의 시퀀스를 생성하고 가장 높은 토큰 로그 확률이 있는 경우 반환
* **repetition_penalty (`float`)** – 반복 패널티에 대한 파라미터, 1.0은 패널티가 없음을 의미하여 Greedy 서치와 동일, 커질수록 다양한 결과를 얻을 수 있으며, 자세한 사항은 [this paper](https://arxiv.org/pdf/1909.05858.pdf)을 참고
* **return_full_text (`bool`)** – 생성된 텍스트에 프롬프트를 추가할지 여부
* **seed (`int`)** – Random sampling seed
* **stop_sequences (`List[str]`)** – `stop_sequences` 가 생성되면 토큰 생성을 중지
* **temperature (`float`)** – logits 분포 모듈화에 사용되는 값
* **top_k (`int`)** – 상위 K개 만큼 가장 높은 확률 어휘 토큰의 수
* **top_p (`float`)** – 1 보다 작게 설정하게 되며, 상위부터 정렬했을 때 가능한 토큰들의 확률을 합산하여 `top_p` 이상의 가장 작은 집합을 유지
* **truncate (`int`)** – 입력 토큰을 지정된 크기로 잘라냄
* **typical_p (`float`)** – typical Decoding 양으로, 자세한 사항은 [Typical Decoding for Natural Language Generation](https://arxiv.org/abs/2202.00666)을 참고
* **watermark (`bool`)** –  [A Watermark for Large Language Models](https://arxiv.org/abs/2301.10226)가 Watermarking
* **decoder_input_details (`bool`)** – decoder input token logprobs와 ids를 반환

In [ ]:
params = {
    "do_sample": False,
    "max_new_tokens": 128,
    "temperature": 0.4,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "presence_penalty": None,
    "eos_token_id": 2,
}

In [ ]:
import json
from inference_lib import KoLLMSageMakerEndpoint, Prompter
ep = KoLLMSageMakerEndpoint(endpoint_name)

In [ ]:
instruction = "다음 글을 알기 쉽게 요약해 주세요."
context = """
대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. 
대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. 
이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. 
특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. 
파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. 
파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.
"""
payload = ep.get_payload(instruction, context, params)

In [ ]:
%%time
generated_text = ep.infer(payload, verbose=True)

In [ ]:
input_text = """
고객님: 안녕하세요, iPhone에 문제가 있습니다.
상담원: 안녕하세요! 무슨 문제인가요?
고객님: 휴대폰이 제대로 충전되지 않고 배터리가 매우 빨리 소모되는 것 같습니다. 다른 충전 케이블과 전원 어댑터를 사용해 보았지만 문제가 지속됩니다.
상담원: 흠, 그렇군요. 몇 가지 문제 해결 단계를 시도해 보겠습니다. 설정, 배터리로 이동하여 배터리 수명을 많이 소모하는 앱이 있는지 확인해 주시겠어요?
고객님: 예, 배터리를 많이 소모하는 앱이 몇 개 있습니다.
상담원: 좋아요, 화면 하단에서 위로 스와이프하여 해당 앱을 강제 종료한 다음 앱을 위로 스와이프하여 닫아 보세요.
고객: 그렇게 했는데도 문제가 여전히 남아 있습니다.
상담원: 네, iPhone의 설정을 기본값으로 재설정해 보겠습니다. 이렇게 해도 데이터가 삭제되지는 않습니다. 설정, 일반, 재설정으로 이동한 다음 모든 설정 재설정을 선택하세요.
고객님: 그렇게 했습니다. 다음은 어떻게 해야 하나요?
상담원: 이제 iPhone을 재시동해 보겠습니다. "밀어서 전원 끄기" 옵션이 표시될 때까지 전원 버튼을 길게 누릅니다. 밀어 전원을 끄고 몇 초간 기다린 다음 iPhone을 다시 켜세요.
고객님: 다시 시작했지만 여전히 제대로 충전되지 않습니다.
상담원: 그렇군요. iPhone에서 진단 테스트를 실행해야 할 것 같습니다. 가까운 Apple Store 또는 공인 서비스 제공업체를 방문하여 iPhone을 점검받으시기 바랍니다.
고객: 예약을 해야 하나요?
상담원: 예. 줄을 서서 기다리지 않으려면 항상 미리 예약하는 것이 가장 좋습니다. 온라인으로 예약하거나 Apple Store 또는 공인 서비스 제공업체에 전화하여 예약할 수 있습니다.
고객님: 수리 비용은 제가 지불해야 하나요?
상담원: iPhone에 보증이 적용되는지 여부에 따라 다릅니다. 보증이 적용되는 경우에는 비용을 지불할 필요가 없습니다. 그러나 보증이 적용되지 않는 경우에는 수리 비용을 지불하셔야 합니다.
고객님: iPhone을 돌려받는 데 얼마나 걸리나요?
상담원: 문제의 심각도에 따라 다르지만 일반적으로 영업일 기준 1~2일이 소요됩니다.
고객: 온라인으로 수리 상태를 추적할 수 있나요?
상담원: 온라인 또는 Apple Store 또는 공인 서비스 제공업체에 전화하여 수리 상태를 추적할 수 있습니다.
고객: 알겠습니다. 도와주셔서 감사합니다.
"""

In [ ]:
%%time
instruction = 'iPhone 충전 문제를 해결하기 위해 고객에게 어떤 문제 해결 단계를 제안하나요?'
payload = ep.get_payload(instruction, input_text, params)
generated_text = ep.infer(payload, verbose=True)

In [ ]:
%%time
instruction = '고객이 iPhone 수리를 위해 가까운 Apple Store 또는 공인 서비스 제공업체에 예약하려면 어떤 조치를 취해야 하나요?'
payload = ep.get_payload(instruction, input_text, params)
generated_text = ep.infer(payload, verbose=True)

In [ ]:
%store endpoint_name

<br>

## 5. Clean Up
---

In [ ]:
!rm -rf src

In [ ]:
# - Delete the end point
sm_client.delete_endpoint(EndpointName=endpoint_name)
# - In case the end point failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

<br>

# References
---

- Model 정보
    - kullm-polyglot-5.8b-v2
        - This model is a parameter-efficient fine-tuned version of EleutherAI/polyglot-ko-5.8b on a KULLM v2
        - https://huggingface.co/nlpai-lab/kullm-polyglot-5.8b-v2        
    - kullm-polyglot-12.8b-v2
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KULLM v2
        - https://huggingface.co/nlpai-lab/kullm-polyglot-12.8b-v2
    - beomi/KoAlpaca-Polyglot-12.8B
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KoAlpaca Dataset v1.1b
        - https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B
    - EleutherAI/polyglot-ko-12.8b
        - Polyglot-Ko-12.8B was trained for 167 billion tokens over 301,000 steps on 256 A100 GPUs with the GPT-NeoX framework. It was trained as an autoregressive language model, using cross-entropy loss to maximize the likelihood of predicting the next token.
        - License: Apache 2.0
        - https://huggingface.co/EleutherAI/polyglot-ko-12.8b      
- 코드
    - [Boto3](https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/pytorch_deploy_large_GPT_model/GPT-J-6B-model-parallel-inference-DJL.ipynb)
    - [Python SDK](https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb)
    - [Kor LLM on SageMaker](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker)
    - [AWS Generative AI Workshop for Korean language](https://github.com/aws-samples/aws-ai-ml-workshop-kr/tree/master/genai)